<a href="https://colab.research.google.com/github/joekelly211/masfi/blob/main/3_predictors_lcluc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports, directories and global functions

In [ ]:
# Define base directory
base_dir = "/gdrive/Shareddrives/masfi"

# Mount Google Drive and set base directory
from google.colab import drive
import os
import sys
drive.mount('/gdrive', force_remount=True)
_path_to_add = os.path.realpath(base_dir)
if _path_to_add not in sys.path:
    sys.path.append(_path_to_add)

Mounted at /gdrive


In [ ]:
# Installs
%%capture
!pip install astropy
!pip install earthengine-api
!pip install geopandas

In [ ]:
!# Reload imports, replacing those in the cache
%load_ext autoreload
%autoreload 2
# Imports
from astropy.convolution import convolve, Gaussian2DKernel
import csv
import ee
import geopandas as gpd
import glob
from google.colab import runtime, userdata
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from os import makedirs, remove
from os.path import exists, join
from osgeo import gdal, gdalconst
from scipy.ndimage import distance_transform_edt
from scipy.ndimage import label, sum as ndi_sum
from shutil import copyfile
from time import sleep

In [ ]:
# 1_areas directories
areas_dir = join(base_dir, "1_areas")
polygons_dir = join(areas_dir, "polygons")
template_dir = join(areas_dir, "template.tif")

# 3_predictors directories
predictors_dir = join(base_dir, "3_predictors")
ee_dir = join(predictors_dir, "earth_engine")
user_upload_dir = join(predictors_dir, "user_upload")
glad_lcluc_dir = join(predictors_dir, 'glad_lcluc')
resampled_dir = join(predictors_dir, "resampled")
continuous_final_dir = join(predictors_dir, "continuous_final")
binary_dir = join(predictors_dir, 'binary')
edge_effects_dir = join(predictors_dir, 'binary_edge_effects')

# 6_scenarios directories
scenario_dir = join(base_dir, "6_scenarios")
scenario_mask_dir = join(scenario_dir, "scenario_masks")

# Create directories
makedirs(ee_dir, exist_ok=True)
makedirs(user_upload_dir, exist_ok=True)
makedirs(glad_lcluc_dir, exist_ok=True)
makedirs(resampled_dir, exist_ok=True)
makedirs(continuous_final_dir, exist_ok=True)
makedirs(binary_dir, exist_ok=True)
makedirs(edge_effects_dir, exist_ok=True)
makedirs(scenario_dir, exist_ok=True)
makedirs(scenario_mask_dir, exist_ok=True)

In [ ]:
# export_array_as_tif function
nodatavalue = -1111111
compress = True
def export_array_as_tif(input_array, output_tif, template=template_dir, nodatavalue=nodatavalue, compress=compress):
  template = gdal.Open(template)
  template_band = template.GetRasterBand(1)
  template_dimensions, template_projection = template.GetGeoTransform(), template.GetProjection()
  if compress: driver = gdal.GetDriverByName("GTiff").Create(output_tif, template_band.XSize, template_band.YSize, bands=1, eType=gdal.GDT_Float32,
                                                options=["COMPRESS=DEFLATE","PREDICTOR=2","ZLEVEL=9"])
  if compress == False: driver = gdal.GetDriverByName("GTiff").Create(output_tif, template_band.XSize, template_band.YSize, bands=1, eType=gdal.GDT_Float32)
  driver.GetRasterBand(1).WriteArray(input_array)
  driver.GetRasterBand(1).SetNoDataValue(nodatavalue)
  driver.SetGeoTransform(template_dimensions)
  driver.SetProjection(template_projection)

# Burn a polygon to raster
def burn_polygon_to_raster(raster_path, polygon_path, fixed=True, fixed_value=1, column_name=None, all_touched=True):
    raster = gdal.Open(raster_path, gdal.GA_Update)
    vector = ogr.Open(polygon_path)
    layer = vector.GetLayer()
    if all_touched: options = ["ALL_TOUCHED=TRUE"]
    else: options = []
    if not fixed: options.append(f"ATTRIBUTE={column_name or layer.GetLayerDefn().GetFieldDefn(0).GetName()}")
    gdal.RasterizeLayer(raster, [1], layer,
                        burn_values=[fixed_value] if fixed else None,
                        options=options)
    raster.FlushCache()
    raster = vector = None

# Download Earth Engine rasters

In [ ]:
# Enable Google Earth Engine API at Google Cloud https://console.cloud.google.com/apis/dashboard
# See here for walkthrough: https://github.com/googlecolab/colabtools/issues/4228#issuecomment-1859068706
# Set project ID under 'secrets' tab on the left with the name 'google_cloud_project'
ee_project = userdata.get('google_cloud_project')

# Authenticate Earth Engine
ee.Authenticate()
ee.Initialize(project=ee_project)

In [ ]:
# Edit this section to change which Earth Engine datasets are downloaded.

# Warning: Earth Engine uses 'nearest neighbour' to resample rasters to the desired extent and resolution before exporting.
# This creates artifacts if the data is continuous, such as DEMs (elevation) or other topographic metrics.
# These should be downloaded from the original source, uploaded to '/user_upload' and resampled in the next section, checking the option for 'bilinear'.

# Check datasets in https://code.earthengine.google.com/ with:
# var assetList = ee.data.listAssets("projects/JRC/TMF/v1_2022/");
# print(assetList);

ee_datasets = [

    {
        "ee_dataset_name": "tmf",
        "ee_dataset_type": "ImageCollection",
        "ee_paths": [
            "projects/JRC/TMF/v1_2023/AnnualChanges",
            "projects/JRC/TMF/v1_2023/AnnualDisruptionObs2023",
            "projects/JRC/TMF/v1_2023/TransitionMap_MainClasses",
            "projects/JRC/TMF/v1_2023/TransitionMap_Subtypes",
            "projects/JRC/TMF/v1_2023/Ndisturb_C2_1982_2022",
        ],
    }
    # {
    #     "ee_dataset_name": "glad",
    #     "ee_dataset_type": "Image",
    #     "ee_paths": [
    #                 # 'projects/glad/GLCLU2020/Forest_gain',
    #                 'projects/glad/GLCLU2020/Forest_height_2000',
    #                 'projects/glad/GLCLU2020/Forest_height_2005',
    #                 'projects/glad/GLCLU2020/Forest_height_2010',
    #                 'projects/glad/GLCLU2020/Forest_height_2015',
    #                 'projects/glad/GLCLU2020/Forest_height_2020',
    #                 'projects/glad/GLCLU2020/Forest_height_disturbance',
    #                 'projects/glad/GLCLU2020/Forest_height_netgain',
    #                 'projects/glad/GLCLU2020/Forest_height_netloss',
    #                 # 'projects/glad/GLCLU2020/Forest_loss',
    #                 # 'projects/glad/GLCLU2020/Forest_type',
    #                 'projects/glad/GLCLU2020/LCLUC',
    #                 'projects/glad/GLCLU2020/LCLUC_2000',
    #                 'projects/glad/GLCLU2020/LCLUC_2020',
    #                 'projects/glad/GLCLU2020/Vegetation_cover_2000',
    #                 'projects/glad/GLCLU2020/Vegetation_cover_2005',
    #                 'projects/glad/GLCLU2020/Vegetation_cover_2010',
    #                 'projects/glad/GLCLU2020/Vegetation_cover_2015',
    #                 'projects/glad/GLCLU2020/Vegetation_cover_2020',
    #                 'projects/glad/GLCLU2020/Vegetation_cover_gain',
    #                 'projects/glad/GLCLU2020/Vegetation_cover_loss',
    #                 'projects/glad/GLCLU2020/Water_2000',
    #                 'projects/glad/GLCLU2020/Water_2005',
    #                 'projects/glad/GLCLU2020/Water_2010',
    #                 'projects/glad/GLCLU2020/Water_2015',
    #                 'projects/glad/GLCLU2020/Water_2020',
    #                 # 'projects/glad/GLCLU2020/Water_dynamics',
    #                 # 'projects/glad/GLCLU2020/Water_dynamics_classes',
    #     ]
    # }
]

In [ ]:
# Verify Earth Engine rasters that will be downloaded
ee_raster_list = []
for ee_dataset in ee_datasets:
    ee_dataset_name = ee_dataset['ee_dataset_name']
    ee_dataset_type = ee_dataset['ee_dataset_type']
    ee_paths = ee_dataset['ee_paths']
    for ee_path in ee_paths:
        if ee_dataset_type == 'ImageCollection':
            ee_image = ee.ImageCollection(ee_path)
            ee_bands = [b['id'] for b in ee_image.getInfo()['features'][0]['bands']]
        else:
            ee_image = ee.Image(ee_path)
            ee_bands = ee_image.bandNames().getInfo()
        for ee_band in ee_bands:
            ee_tif_filename = f"{ee_dataset_name}_{ee_path.split('/')[-1]}_{ee_band}.tif"
            ee_raster_list.append(ee_tif_filename)

ee_raster_list = list(reversed(ee_raster_list))
ee_raster_list

['tmf_Ndisturb_C2_1982_2022_y2022.tif',
 'tmf_Ndisturb_C2_1982_2022_y2021.tif',
 'tmf_Ndisturb_C2_1982_2022_y2020.tif',
 'tmf_Ndisturb_C2_1982_2022_y2019.tif',
 'tmf_Ndisturb_C2_1982_2022_y2018.tif',
 'tmf_Ndisturb_C2_1982_2022_y2017.tif',
 'tmf_Ndisturb_C2_1982_2022_y2016.tif',
 'tmf_Ndisturb_C2_1982_2022_y2015.tif',
 'tmf_Ndisturb_C2_1982_2022_y2014.tif',
 'tmf_Ndisturb_C2_1982_2022_y2013.tif',
 'tmf_Ndisturb_C2_1982_2022_y2012.tif',
 'tmf_Ndisturb_C2_1982_2022_y2011.tif',
 'tmf_Ndisturb_C2_1982_2022_y2010.tif',
 'tmf_Ndisturb_C2_1982_2022_y2009.tif',
 'tmf_Ndisturb_C2_1982_2022_y2008.tif',
 'tmf_Ndisturb_C2_1982_2022_y2007.tif',
 'tmf_Ndisturb_C2_1982_2022_y2006.tif',
 'tmf_Ndisturb_C2_1982_2022_y2005.tif',
 'tmf_Ndisturb_C2_1982_2022_y2004.tif',
 'tmf_Ndisturb_C2_1982_2022_y2003.tif',
 'tmf_Ndisturb_C2_1982_2022_y2002.tif',
 'tmf_Ndisturb_C2_1982_2022_y2001.tif',
 'tmf_Ndisturb_C2_1982_2022_y2000.tif',
 'tmf_Ndisturb_C2_1982_2022_y1999.tif',
 'tmf_Ndisturb_C2_1982_2022_y1998.tif',


In [ ]:
# Check whether using mydrive as opposed to a shared gdrive
mydrive_mounted = False

# Earth Engine download progress
ee_progress_index = 0
ee_progress_label = widgets.Label(f"Earth Engine download progress: {ee_progress_index}/{len(ee_raster_list)}")
display(ee_progress_label)

# Load template and set Earth Engine geometry
template_polygon_dir = join(polygons_dir, 'template.gpkg')
template_area = gpd.read_file(template_polygon_dir)["geometry"].iloc[0]
template_coords = list(gpd.read_file(template_polygon_dir)["geometry"].iloc[0].exterior.coords)
ee_geometry = ee.Geometry.Polygon(template_coords)

# Download Earth Engine datasets
for ee_dataset in ee_datasets:
    ee_dataset_name = ee_dataset['ee_dataset_name']
    ee_dataset_type = ee_dataset['ee_dataset_type']
    ee_paths = ee_dataset['ee_paths']
    # Loop through Earth Engine paths
    for ee_path in ee_paths:
        # identify bands
        if ee_dataset_type == 'ImageCollection':
            ee_image = ee.ImageCollection(ee_path)
            ee_bands = [b['id'] for b in ee_image.getInfo()['features'][0]['bands']]
        elif ee_dataset_type == 'Image':
            ee_image = ee.Image(ee_path)
            ee_bands = ee_image.bandNames().getInfo()
        # Loop through bands
        for ee_band in reversed(ee_bands):
            # Set filename and directory of downloaded raster and check if exists
            ee_tif_filename = f"{ee_dataset_name}_{ee_path.split('/')[-1]}_{ee_band}.tif"
            ee_tif_dir = join(ee_dir, ee_tif_filename)
            ee_temp_dir = join("/gdrive/MyDrive", ee_tif_filename)
            if mydrive_mounted: ee_temp_dir = join("/content/drive/MyDrive/", ee_tif_filename)
            # Check if temporary raster exists and needs copying
            if exists(ee_temp_dir):
              copyfile(ee_temp_dir, ee_tif_dir)
              remove(ee_temp_dir)
            # Check if copied raster exists, and if not download from Earth Engine.
            if not exists(ee_tif_dir):
              if ee_dataset_type == 'ImageCollection':
                image_selected = ee_image.qualityMosaic(ee_band).select([ee_band])
                resolution = ee_image.first().projection().nominalScale().getInfo()
              if ee_dataset_type == 'Image':
                image_selected = ee_image.select([ee_band])
                resolution = ee_image.select(0).projection().nominalScale().getInfo()
              ee_task = ee.batch.Export.image.toDrive(image=image_selected.toFloat(),
                                                    description=ee_tif_filename[:-4],
                                                    scale=resolution,
                                                    region=ee_geometry,
                                                    maxPixels=10000000000,
                                                    fileNamePrefix=ee_tif_filename[:-4],
                                                    crs='EPSG:4326',
                                                    fileFormat='GeoTIFF')
              ee_task.start()
              # Check whether the raster has downloaded yet
              while not exists(ee_temp_dir):
                  ee_task_status = ee_task.status()
                  # If the task is completed, continue
                  if ee_task_status["state"] == 'COMPLETED': break
                  # If it has failed or been cancelled, show an error
                  elif ee_task_status['state'] == 'FAILED' or ee_task_status['state'] == 'CANCELLED':
                      print(f"{ee_tif_filename}:{ee_task_status['error_message']}")
                      try: remove(ee_temp_dir)
                      except: pass
                      break
                  sleep(1)
              # Copy the raster to intended directory and remove the temporary raster
              while not exists(ee_temp_dir):
                sleep(1)
              copyfile(ee_temp_dir, ee_tif_dir)
              remove(ee_temp_dir)
            # Update Earth Engine download progress
            ee_progress_index += 1
            ee_progress_label.value = f"Earth Engine download progress: {ee_progress_index}/{len(ee_raster_list)}"

# Check Earth Engine tasks here: https://code.earthengine.google.com/tasks

Label(value='Earth Engine download progress: 0/78')

# GLAD LCLUC

In [ ]:
# LCLUC contains several land cover and land use types, each with continuous metrics.
# This splits them into categories for better modelling, based on the legend:
# https://glad.umd.edu/sites/default/files/legend_0.xlsx
# Should do before resampling.

lcluc_dict = {
    'terra_vegetation_cover_percent': (0, 24),
    'terra_stable_tree_m': (25, 48),
    'wetland_vegetation_cover_percent': (100, 124),
    'wetland_stable_tree_m': (125, 148),
    'open_surface_water_percent_of_year': (200, 207),
    'snow_ice': (241, 241),
    'cropland': (244, 244),
    'built_up': (250, 250),
    'ocean': (254, 254),
}

for lcluc_raster in os.listdir(ee_dir):
  if 'LCLUC' in lcluc_raster:
    lcluc_path = join(ee_dir, lcluc_raster)
    luluc_array = gdal.Open(lcluc_path).ReadAsArray()
    for key, (lower, upper) in lcluc_dict.items():
        split_luluc_filename = f"{lcluc_raster[:-4]}_{key}.tif"
        split_luluc_filename_binary = f"{lcluc_raster[:-4]}_{key}_binary.tif"
        split_luluc_dir = join(glad_lcluc_dir, split_luluc_filename)
        split_luluc_dir_binary = join(glad_lcluc_dir, split_luluc_filename_binary)
        if not exists(split_luluc_dir) and not exists(split_luluc_dir_binary):
          split_luluc_mask = np.logical_and(luluc_array >= lower, luluc_array <= upper)
          split_luluc_array = np.where(split_luluc_mask, luluc_array, 0) # outside the range set to 0
          non_zero_percentage = np.count_nonzero(split_luluc_array) / split_luluc_array.size * 100
          if non_zero_percentage >= 0.1:
            # Check if there's only one unique non-zero value, and convert to a 1-0 binary raster if true
            unique_non_zero_values = np.unique(split_luluc_array[split_luluc_array > 0])
            if len(unique_non_zero_values) == 1:
                split_luluc_array = np.where(split_luluc_array > 0, 1, 0)
                split_luluc_dir = split_luluc_dir_binary
            export_array_as_tif(split_luluc_array, split_luluc_dir, template=lcluc_path)

# Resample EE rasters

In [ ]:
# Create dictionary of all tifs in Earth Engine and user upload directory
resample_dict = {}
for resample_raster in os.listdir(ee_dir):
    resample_dict.update({f'{resample_raster}':"'categorical'"})
for resample_raster in os.listdir(user_upload_dir):
    resample_dict.update({f'{resample_raster}':"'categorical'"})
for resample_raster in os.listdir(glad_lcluc_dir):
    resample_dict.update({f'{resample_raster}':"'continuous'"})
resample_dict = {key: value for key, value in sorted(resample_dict.items())}

# Select rasters for resampling and verify data type (categorical or continuous)
print("selected_original_rasters = {")
for key, value in resample_dict.items():
    print(f'"{key}": {value},')
print("}")

selected_original_rasters = {
"tmf_AnnualChanges_Dec1990.tif": 'categorical',
"tmf_AnnualChanges_Dec1991.tif": 'categorical',
"tmf_AnnualChanges_Dec1992.tif": 'categorical',
"tmf_AnnualChanges_Dec1993.tif": 'categorical',
"tmf_AnnualChanges_Dec1994.tif": 'categorical',
"tmf_AnnualChanges_Dec1995.tif": 'categorical',
"tmf_AnnualChanges_Dec1996.tif": 'categorical',
"tmf_AnnualChanges_Dec1997.tif": 'categorical',
"tmf_AnnualChanges_Dec1998.tif": 'categorical',
"tmf_AnnualChanges_Dec1999.tif": 'categorical',
"tmf_AnnualChanges_Dec2000.tif": 'categorical',
"tmf_AnnualChanges_Dec2001.tif": 'categorical',
"tmf_AnnualChanges_Dec2002.tif": 'categorical',
"tmf_AnnualChanges_Dec2003.tif": 'categorical',
"tmf_AnnualChanges_Dec2004.tif": 'categorical',
"tmf_AnnualChanges_Dec2005.tif": 'categorical',
"tmf_AnnualChanges_Dec2006.tif": 'categorical',
"tmf_AnnualChanges_Dec2007.tif": 'categorical',
"tmf_AnnualChanges_Dec2008.tif": 'categorical',
"tmf_AnnualChanges_Dec2009.tif": 'categorical',
"tmf_Annua

In [ ]:
selected_original_rasters = {
"tmf_AnnualChanges_Dec1990.tif": 'categorical',
"tmf_AnnualChanges_Dec1991.tif": 'categorical',
"tmf_AnnualChanges_Dec1992.tif": 'categorical',
"tmf_AnnualChanges_Dec1993.tif": 'categorical',
"tmf_AnnualChanges_Dec1994.tif": 'categorical',
"tmf_AnnualChanges_Dec1995.tif": 'categorical',
"tmf_AnnualChanges_Dec1996.tif": 'categorical',
"tmf_AnnualChanges_Dec1997.tif": 'categorical',
"tmf_AnnualChanges_Dec1998.tif": 'categorical',
"tmf_AnnualChanges_Dec1999.tif": 'categorical',
"tmf_AnnualChanges_Dec2000.tif": 'categorical',
"tmf_AnnualChanges_Dec2001.tif": 'categorical',
"tmf_AnnualChanges_Dec2002.tif": 'categorical',
"tmf_AnnualChanges_Dec2003.tif": 'categorical',
"tmf_AnnualChanges_Dec2004.tif": 'categorical',
"tmf_AnnualChanges_Dec2005.tif": 'categorical',
"tmf_AnnualChanges_Dec2006.tif": 'categorical',
"tmf_AnnualChanges_Dec2007.tif": 'categorical',
"tmf_AnnualChanges_Dec2008.tif": 'categorical',
"tmf_AnnualChanges_Dec2009.tif": 'categorical',
"tmf_AnnualChanges_Dec2010.tif": 'categorical',
"tmf_AnnualChanges_Dec2011.tif": 'categorical',
"tmf_AnnualChanges_Dec2012.tif": 'categorical',
"tmf_AnnualChanges_Dec2013.tif": 'categorical',
"tmf_AnnualChanges_Dec2014.tif": 'categorical',
"tmf_AnnualChanges_Dec2015.tif": 'categorical',
"tmf_AnnualChanges_Dec2016.tif": 'categorical',
"tmf_AnnualChanges_Dec2017.tif": 'categorical',
"tmf_AnnualChanges_Dec2018.tif": 'categorical',
"tmf_AnnualChanges_Dec2019.tif": 'categorical',
"tmf_AnnualChanges_Dec2020.tif": 'categorical',
"tmf_AnnualChanges_Dec2021.tif": 'categorical',
"tmf_AnnualChanges_Dec2022.tif": 'categorical',
"tmf_AnnualChanges_Dec2023.tif": 'categorical',
"tmf_AnnualDisruptionObs2023_y2023.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1982.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1983.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1984.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1985.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1986.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1987.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1988.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1989.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1990.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1991.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1992.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1993.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1994.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1995.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1996.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1997.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1998.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y1999.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2000.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2001.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2002.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2003.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2004.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2005.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2006.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2007.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2008.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2009.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2010.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2011.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2012.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2013.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2014.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2015.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2016.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2017.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2018.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2019.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2020.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2021.tif": 'categorical',
"tmf_Ndisturb_C2_1982_2022_y2022.tif": 'categorical',
"tmf_TransitionMap_MainClasses_TransitionMap_MainClasses.tif": 'categorical',
"tmf_TransitionMap_Subtypes_TransitionMap_Subtypes.tif": 'categorical',
}

In [ ]:
# Set resample algorithms for different raster types
# See https://gdal.org/programs/gdalwarp.html
categorical_alg = 'near'
continuous_alg = 'bilinear'

template = gdal.Open(template_dir)
template_dimensions = template.GetGeoTransform()
xres, yres = template_dimensions[1], -template_dimensions[5]
xmin = template_dimensions[0]
ymin = template_dimensions[3] - template.RasterYSize * yres
xmax = xmin + template.RasterXSize * xres
ymax = template_dimensions[3]

# Resample progress
resample_progress_index = 0
resample_progress_label = widgets.Label(f"Resample progress: {resample_progress_index}/{len(selected_original_rasters.items())}")
display(resample_progress_label)

# Iterate over selected rasters
for original_raster_name, data_type in selected_original_rasters.items():
  resampled_raster_dir = join(resampled_dir, original_raster_name)
  if not exists(resampled_raster_dir):
    original_raster_dir = join(ee_dir, original_raster_name)
    if not exists(original_raster_dir): original_raster_dir = join(user_upload_dir, original_raster_name)
    if not exists(original_raster_dir): original_raster_dir = join(glad_lcluc_dir, original_raster_name)
    # Set resample type
    if data_type == 'categorical': resample_alg = categorical_alg
    if data_type == 'continuous': resample_alg = continuous_alg
    src = gdal.Warp(
        resampled_raster_dir,
        original_raster_dir,
        xRes=xres, yRes=yres,
        outputBounds=(xmin, ymin, xmax, ymax),
        resampleAlg=resample_alg,
        outputType=gdalconst.GDT_Float32)
    # Compress and close
    driver = gdal.GetDriverByName("GTiff")
    src = driver.CreateCopy(resampled_raster_dir, src, 0, options=["COMPRESS=DEFLATE","PREDICTOR=2","ZLEVEL=9"])
    src = None
  # Update resample progress
  resample_progress_index += 1
  resample_progress_label.value = f"Resample progress: {resample_progress_index}/{len(selected_original_rasters.items())}"

In [ ]:
# Determine continous predictor precision

override_max_unique_values = False
max_unique_values = 5000 # Should be >=10

if override_max_unique_values == False:
  dem_base_path = join(areas_dir, "base_dem.tif")
  dem_base_array = gdal.Open(dem_base_path).ReadAsArray()
  max_unique_values = int(np.ptp(dem_base_array)) # Precision based on elevation variance
resampled_precision_dict = {}

for resampled_predictor, resample_type in selected_original_rasters.items():
  if resample_type == 'continuous':
    resampled_predictor_path = join(resampled_dir, resampled_predictor)
    print(f"Reading {resampled_predictor}...")
    # Read raster as array
    resampled_predictor_array = gdal.Open(resampled_predictor_path).ReadAsArray()
    # Convert 'nodata' values to nan
    resampled_predictor_array[resampled_predictor_array == nodatavalue] = np.nan
    resampled_predictor_array_masked = np.ma.array(resampled_predictor_array, mask=np.isnan(resampled_predictor_array))
    # Count unique values in raster
    unique_values = len(np.unique(resampled_predictor_array_masked))
    print(f"There are {unique_values} unique values in {resampled_predictor}")
    # Generate histogram from 100,000 random points
    random_selection = np.random.choice(resampled_predictor_array_masked.ravel(), size = 100_000, replace = False)
    _ = plt.hist(random_selection, bins='auto')  # arguments are passed to np.histogram
    plt.title(f"{resampled_predictor}")
    plt.show()
    # Remove 0 values for log10
    resampled_predictor_array_masked[resampled_predictor_array_masked == 0] = np.nan
    resampled_predictor_array_masked = np.ma.array(resampled_predictor_array, mask=np.isnan(resampled_predictor_array))
    # Create log10 array for determining positions for rounding
    array_log10 = np.log10(abs(resampled_predictor_array_masked))
    place_value_decimal = int(abs(np.min(array_log10)))
    place_value_integer = int(0 - np.max(array_log10))
    # Iterate down precision levels to determine optimal number of unique values
    min_starting_precision = len(str(max_unique_values))
    for precision in reversed(range(place_value_integer, max(min_starting_precision, place_value_decimal +1))):
      rounded_array = np.round(resampled_predictor_array, decimals=precision)
      round_unique_values = len(np.unique(rounded_array))
      optimal_precision = None
      if round_unique_values <= max_unique_values:
        optimal_precision = precision
        print(f"The optimal precison for {resampled_predictor} is {optimal_precision}, with {round_unique_values} unique values.")
        resampled_precision_dict.update({f'{resampled_predictor}':f'{optimal_precision}'})
        break
    if optimal_precision == None: print("There's a problem with setting precision.")
    print("___________________\n")

print("Dictionary for optimal rounding values:")
resampled_precision_dict

precision_dict_csv_path = join(resampled_dir, 'rounding_dictionary.csv')
# Save rounding dictionary to CSV
with open(precision_dict_csv_path, 'w', newline='') as precision_dict_csv:
    writer = csv.writer(precision_dict_csv)
    writer.writerow(resampled_precision_dict.keys())
    writer.writerow(resampled_precision_dict.values())

Dictionary for optimal rounding values:


In [ ]:
# Open rounding dictionary and verify
with open(precision_dict_csv_path, 'r') as file:
    keys, values = list(csv.reader(file))
    topo_precision_dict = dict(zip(keys, values))

# Verify precision and correct if necessary
print("topo_precision_dict = {")
for key, value in topo_precision_dict.items():
    print(f'"{key}": {value},')
print("}")

topo_precision_dict = {
}


In [ ]:
topo_precision_dict = {
"agbd_2022.tif": 0,
"agbd_diff_disturbance_1990_2022.tif": 0,
"agbd_diff_disturbance_before_1990.tif": 0,
}

In [ ]:
# Set smoothing kernel
kernel = Gaussian2DKernel(x_stddev=1, y_stddev=1)

# Continuous progress
continuous_progress_index = 0
continuous_progress_label = widgets.Label(f"Continuous progress: {continuous_progress_index}/{len(topo_precision_dict.items())}")
display(continuous_progress_label)

# Iterate over selected continuous rasters
for continuous, precision in topo_precision_dict.items():
  cont_raster_resampled_path = join(resampled_dir, continuous)
  cont_raster_resampled_array = gdal.Open(cont_raster_resampled_path).ReadAsArray()
  # Convert nodata values to 0
  cont_raster_resampled_array[cont_raster_resampled_array == nodatavalue] = 0
  # Set path and check if exists
  cont_raster_unsmoothed_filename = f"{continuous[:-4]}_unsmooth.tif"
  cont_raster_unsmoothed_path = join(continuous_final_dir, cont_raster_unsmoothed_filename)
  if not exists(cont_raster_unsmoothed_path):
    # Round and export unsmoothed continuous raster
    cont_raster_unsmoothed_rounded = np.round(cont_raster_resampled_array, decimals=int(precision))
    export_array_as_tif(cont_raster_unsmoothed_rounded, cont_raster_unsmoothed_path)
  # Smooth using 2D spatial convolution
  cont_raster_smoothed_filename = f"{continuous[:-4]}_smooth.tif"
  cont_raster_smoothed_path = join(continuous_final_dir, cont_raster_smoothed_filename)
  if not exists(cont_raster_smoothed_path):
    cont_raster_smoothed = convolve(cont_raster_resampled_array, kernel, boundary='extend')
    # Round and export smoothed continuous raster
    cont_raster_smoothed_rounded = np.round(cont_raster_smoothed, decimals=int(precision))
    export_array_as_tif(cont_raster_smoothed_rounded, cont_raster_smoothed_path)
  # Update continuous progress
  continuous_progress_index += 1
  continuous_progress_label.value = f"Continuous progress: {continuous_progress_index}/{len(topo_precision_dict.items())}"

# TMF binary predictors

In [ ]:
# Check TMF data users guide for classification. https://forobs.jrc.ec.europa.eu/static/tmf/TMF_DataUsersGuide.pdf

cell_size_x = gdal.Open(join(areas_dir, 'cell_size_x.tif')).ReadAsArray()
cell_size_y = gdal.Open(join(areas_dir, 'cell_size_y.tif')).ReadAsArray()
cell_size_ha = np.mean(cell_size_x) * np.mean(cell_size_y) / 10_000
sieve_size = int(np.ceil(0.5/cell_size_ha)) # Removes all forest patches smaller than 0.5 ha
print(f"Forest binary sieve size (>0.5 ha) is {sieve_size} pixels.")

# Generate list of valid TMF rasters to convert to binary
binary_list = []
for resampled_raster in os.listdir(resampled_dir):
  # Verify these are in the filenames
  if 'DisruptionObs' in resampled_raster or 'AnnualChanges' in resampled_raster or 'Ndisturb' in resampled_raster:
    # Verify this is the position of the year in the filename
    year = resampled_raster[-8:-4]  # Data prior to 1990 is poor
    if int(year) >= 1990: binary_list.append(resampled_raster)

# Binary progress
binary_progress_index = 0
binary_progress_label = widgets.Label(f"Binary progress: {binary_progress_index}/{len(binary_list)}")
display(binary_progress_label)

for resampled_raster in binary_list:
  year = resampled_raster[-8:-4]
  # Forest binary
  if 'AnnualChanges' in resampled_raster:
    forest_binary_path = join(binary_dir, f"forest_binary_{year}.tif")
    if not exists(forest_binary_path):
      ac_raster_path = join(resampled_dir, resampled_raster)
      ac_array = gdal.Open(ac_raster_path).ReadAsArray()
      # Set classes 1 & 2 as 1, all else as 0
      forest_binary_array = np.where((ac_array == 1) | (ac_array == 2), 1, 0)

      # Sieve to 0.5 ha, using 8-connectedness (3, 3)
      fb_array_labelled, fb_array_features = label(forest_binary_array, structure=np.ones((3, 3)))
      # Determine the size of each patch
      fb_array_sizes = ndi_sum(forest_binary_array, fb_array_labelled, range(fb_array_features + 1))
      # Create a mask to remove patches smaller than the threshold
      fb_array_mask_sizes = fb_array_sizes >= sieve_size
      fb_array_mask_sizes[0] = 0 # Ensure non-forest (0) is excluded
      fb_array_mask = fb_array_mask_sizes[fb_array_labelled]
      # Apply the mask to the forest binary array and export
      fb_array_sieved = forest_binary_array * fb_array_mask
      export_array_as_tif(fb_array_sieved, forest_binary_path)

  # Disturbance binary
  if 'DisruptionObs' in resampled_raster or 'Ndisturb' in resampled_raster:
    disturbance_binary_path = join(binary_dir, f"disturbance_binary_{year}.tif")
    if not exists(disturbance_binary_path):
      ac_raster_path = glob.glob(f"{resampled_dir}/*AnnualChanges*{year}*")
      ac_array = gdal.Open(ac_raster_path[0]).ReadAsArray()
      do_raster_path = join(resampled_dir, resampled_raster)
      do_array = gdal.Open(do_raster_path).ReadAsArray()
      # Set all disruption events to '1' if they're not classed as undisturbed forest or water in AnnualChanges
      disturbance_binary_array = np.where((do_array >= 1) & ((ac_array != 1) & (ac_array != 5)), 1, 0)
      export_array_as_tif(disturbance_binary_array, disturbance_binary_path)

  # Update binary progress
  binary_progress_index += 1
  binary_progress_label.value = f"Binary progress: {binary_progress_index}/{len(binary_list)}"

Forest binary sieve size (>0.5 ha) is 6 pixels.


Label(value='Binary progress: 0/68')

In [ ]:
# Extract mangrove binary (optional)
extract_mangrove_binary = True

if extract_mangrove_binary:
  mangrove_binary_path = join(binary_dir, "mangrove_binary.tif")
  if not exists(mangrove_binary_path):
    # Open subtypes array
    for resampled_raster in os.listdir(resampled_dir):
      if 'Subtypes' in resampled_raster:
        subtypes_raster_path = join(resampled_dir, resampled_raster)
        subtypes_raster_array = gdal.Open(subtypes_raster_path).ReadAsArray()

    # Open oldest available forest binary raster for full mangrove extent (1990)
    forest_binary_1990_path = join(binary_dir, "forest_binary_1990.tif")
    forest_binary_1990_array = gdal.Open(forest_binary_1990_path).ReadAsArray()

    mangrove_binary_array = np.logical_and(forest_binary_1990_array == 1,
        np.logical_or(subtypes_raster_array == 12,(subtypes_raster_array >= 61) & (subtypes_raster_array <= 69))
    )

    # Calculate the percentage of forest pixels that are mangrove
    forest_1990_pixels = np.sum(forest_binary_1990_array)
    mangrove_pixels = np.sum(mangrove_binary_array)
    if mangrove_pixels > 0:
      mangrove_percent = (mangrove_pixels / forest_1990_pixels) * 100
      export_array_as_tif(mangrove_binary_array, mangrove_binary_path)
      print(f"Number of mangrove pixels: {mangrove_pixels}")
      print(f"Percentage of 1990 forest pixels that are mangrove: {mangrove_percent:.2f}%")
    else: print("There are no mangrove pixels in the template area.")
  else: print("A mangrove binary raster already exists. Delete it to generate a new one.")

# LU polygon binary predictors

In [ ]:
# Selected 'land use' polygons.
# Creating a 'complete recovery' or 'complete restoration' scenario requires ONE of these as a proxy.
# This can be multiple combined PAs / polygons that have no or minimal history of human disturbance.

polygons_to_exclude = ['template.gpkg', 'project_area.gpkg', 'project_area_buffered_bbox.gpkg', 'gedi_area.gpkg', 'project_area_inverse.gpkg', 'gedi_area_inverse.gpkg']
print("lu_polygons = [")
for polygon in os.listdir(polygons_dir):
  if polygon not in polygons_to_exclude:
    print(f"'{polygon}',")
print("]")

lu_polygons = [
'peninsular_malaysia.gpkg',
'lu_yong.gpkg',
'lu_yong_lipis.gpkg',
'lu_berkelah_jerantut.gpkg',
'lu_tekai_tembeling.gpkg',
'lu_ais.gpkg',
'lu_pa_taman_negara_krau.gpkg',
'lu_berekelah_kuantan.gpkg',
'lu_tekam.gpkg',
'lu_berkelah_temerloh.gpkg',
'lu_remen_chereh.gpkg',
]


In [ ]:
lu_polygons = [
'peninsular_malaysia.gpkg',
'lu_yong.gpkg',
'lu_yong_lipis.gpkg',
'lu_berkelah_jerantut.gpkg',
'lu_tekai_tembeling.gpkg',
'lu_ais.gpkg',
'lu_pa_taman_negara_krau.gpkg',
'lu_berekelah_kuantan.gpkg',
'lu_tekam.gpkg',
'lu_berkelah_temerloh.gpkg',
'lu_remen_chereh.gpkg',
]

# Convert all template values to 'nodata' in preparation
template_tif = gdal.Open(template_dir)
template_mask_array = gdal.Open(template_dir).ReadAsArray()
template_mask_array[template_mask_array != None] = 0

for lu_polygon in lu_polygons:
  lu_binary_name = f"{lu_polygon[:-5]}_binary.tif"
  lu_binary_path = join(binary_dir, lu_binary_name)
  if not exists(lu_binary_path):
    lu_polygon_path = join(polygons_dir, lu_polygon)
    export_array_as_tif(template_mask_array, lu_binary_path)
    # Burn the value '1' where it overlaps with the project area polygon
    burn_polygon_to_raster(lu_binary_path, lu_polygon_path, fixed=True, fixed_value=1, all_touched=False)
    print(f"{lu_binary_name} has been created.")
  else: print(f"{lu_binary_name} already exists.")

peninsular_malaysia_binary.tif has been created.
lu_yong_binary.tif has been created.
lu_yong_lipis_binary.tif has been created.
lu_berkelah_jerantut_binary.tif has been created.
lu_tekai_tembeling_binary.tif has been created.
lu_ais_binary.tif has been created.
lu_pa_taman_negara_krau_binary.tif has been created.
lu_berekelah_kuantan_binary.tif has been created.
lu_tekam_binary.tif has been created.
lu_berkelah_temerloh_binary.tif has been created.
lu_remen_chereh_binary.tif has been created.


# Binary masks

In [ ]:
# Generate masks for later scenario predictions, e.g. so outputs only show forest.
mask_type_list = []
for binary in os.listdir(binary_dir):
    mask_type = binary.split('_')[0]
    if mask_type not in mask_type_list:
        mask_type_list.append(mask_type)

print("mask_types = [")
for mask_type in mask_type_list:
    print(f"'{mask_type}',")
print("]")

mask_types = [
'forest',
'disturbance',
]


In [ ]:
mask_types = [
'forest',
# 'lu',
# 'disturbance',
]

# Create list of binary rasters to mask
binary_mask_list = []
for mask_type in mask_types:
  for binary in os.listdir(binary_dir):
    if mask_type in binary:
      binary_mask_list.append(binary)

# Binary progress
mask_progress_index = 0
mask_progress_label = widgets.Label(f"Binary progress: {mask_progress_index}/{len(binary_mask_list)}")
display(mask_progress_label)

# Create masks from the selected binary raster type
for mask_type in mask_types:
  for binary in binary_mask_list:
    binary_path = join(binary_dir, binary)
    try: year = str(int(binary[-8:-4])) # Check for year
    except: year = None
    mask_raster_path = join(scenario_mask_dir, f"mask_{mask_type}_{year}.tif")
    if not exists(mask_raster_path):
        binary_raster = gdal.Open(binary_path)
        binary_array = gdal.Open(binary_path).ReadAsArray()
        mask_array = np.where(binary_array == 0, nodatavalue, 1)
        export_array_as_tif(mask_array, mask_raster_path)
        print(f"A mask raster has been created: {mask_raster_path}")
    else: print(f"A mask raster already exists at: {mask_raster_path}")
    # Update mask progress
    mask_progress_index += 1
    mask_progress_label.value = f"Binary progress: {mask_progress_index}/{len(binary_mask_list)}"

Label(value='Binary progress: 0/34')

A mask raster has been created: /gdrive/Shareddrives/masfi/6_scenarios/scenario_masks/mask_forest_1990.tif
A mask raster has been created: /gdrive/Shareddrives/masfi/6_scenarios/scenario_masks/mask_forest_1991.tif
A mask raster has been created: /gdrive/Shareddrives/masfi/6_scenarios/scenario_masks/mask_forest_1992.tif
A mask raster has been created: /gdrive/Shareddrives/masfi/6_scenarios/scenario_masks/mask_forest_1993.tif
A mask raster has been created: /gdrive/Shareddrives/masfi/6_scenarios/scenario_masks/mask_forest_1994.tif
A mask raster has been created: /gdrive/Shareddrives/masfi/6_scenarios/scenario_masks/mask_forest_1995.tif
A mask raster has been created: /gdrive/Shareddrives/masfi/6_scenarios/scenario_masks/mask_forest_1996.tif
A mask raster has been created: /gdrive/Shareddrives/masfi/6_scenarios/scenario_masks/mask_forest_1997.tif
A mask raster has been created: /gdrive/Shareddrives/masfi/6_scenarios/scenario_masks/mask_forest_1998.tif
A mask raster has been created: /gdri

# Binary predictor edge effects

In [ ]:
# Set smoothing kernel
kernel = Gaussian2DKernel(x_stddev=3, y_stddev=3)
# Set precision
precision = 2

binary_list = []
for binary_raster in os.listdir(binary_dir) + os.listdir(resampled_dir):
  if "binary" in binary_raster:
    binary_list.append(binary_raster)

# Edge effect progress
edge_effect_progress_index = 0
edge_effect_progress_label = widgets.Label(f"Edge effect progress: {edge_effect_progress_index}/{len(binary_list)}")
display(edge_effect_progress_label)

for binary_raster in binary_list:
  if "binary" in binary_raster:
    edge_effects_filename = binary_raster.replace('binary', 'with_edge_effects')
    edge_effects_path = join(edge_effects_dir, edge_effects_filename)
    if not exists(edge_effects_path):
      binary_raster_path = join(binary_dir, binary_raster)
      if not exists(binary_raster_path): binary_raster_path = join(resampled_dir, binary_raster)
      binary_array = gdal.Open(binary_raster_path).ReadAsArray()
      # Reclassify for binary differentiation after proximity conversion
      differentiator_array = binary_array.copy()
      differentiator_array[differentiator_array == 1] = 10
      # Positive proximity
      positive_distances = distance_transform_edt(binary_array == 0) # target pixels
      positive_proximity_array = np.where(positive_distances > 2, 0, positive_distances) # max distance 2
      # Negative proximity
      negative_distances = distance_transform_edt(binary_array == 1) # target pixels
      negative_proximity_array = np.where(negative_distances > 2, 0, negative_distances) # max distance 2
      # Sum proximities and differentiator
      pixel_prox_summed =  differentiator_array + positive_proximity_array + negative_proximity_array
      # Reclassify for better semantic understanding of pixel proximity
      pixel_prox_reclassed = pixel_prox_summed.copy()
      pixel_prox_reclass_table = [(0, 0, -4), (1, 1, -1), (1.4, 1.5, -2), (2, 2, -3), (10, 10, 3), (11, 11, 0), (11.4, 11.5, 1), (12, 12, 2)]
      for min_value, max_value, new_value in pixel_prox_reclass_table:
        pixel_prox_reclassed[(pixel_prox_reclassed >= min_value) & (pixel_prox_reclassed <= max_value)] = new_value
      # Smooth binary array using 2D convolution
      binary_smoothed = convolve(binary_array, kernel, boundary='extend')
      # Sum pixel proximity and smoothed binary array
      edge_effects_array = np.round(pixel_prox_reclassed + binary_smoothed, precision)
      # Export edge effects predictors
      export_array_as_tif(edge_effects_array, edge_effects_path)

  # Update binary progress
  edge_effect_progress_index += 1
  edge_effect_progress_label.value = f"Edge effect progress: {edge_effect_progress_index}/{len(binary_list)}"

Label(value='Edge effect progress: 0/79')

# Disconnect runtime

In [ ]:
# Useful for stopping background execution
runtime.unassign()